# Prompt engineering

OpenAI promp engineering for using API requests to extract features from the news collected.

Using openai=2.1.0

Using structured output with BaseModel


In [9]:
from __future__ import annotations
from typing import List, Optional, Literal
from pydantic import BaseModel, Field
import pandas as pd
pd.options.display.max_columns = None
from pandas import json_normalize

from openai import OpenAI

project_path = "C:/Users/santt/Desktop/DataMining_UBA/5-text_mining/nlp_dmuba/"

### Loading dataset

In [10]:
dataset_file_path = project_path+"1-Scraping/dataset_consolidado/df.parquet" 
df = pd.read_parquet(dataset_file_path)

In [11]:
df.head()

,diario,fecha,titulo,contenido,url,seccion
0,Ámbito Financiero,2025-01-01,La reflexión de Javier Milei: el pronóstico pa...,Arrancó el 2025 y el presidente Javier Milei h...,https://www.ambito.com/politica/la-reflexion-j...,politica
1,Ámbito Financiero,2025-01-01,Wall Street apuesta por Wall Street: los más o...,Wall Street cerró el año con leves tomas de ga...,https://www.ambito.com/economia/wall-street-ap...,economia
2,Ámbito Financiero,2025-01-01,Karina Milei a los libertarios por Año Nuevo: ...,"La secretaria general de la Presidencia, Karin...",https://www.ambito.com/politica/karina-milei-l...,politica
3,Ámbito Financiero,2025-01-01,"El mensaje de Javier Milei por Año Nuevo: ""Se ...",El presidente Javier Milei compartió un mensaj...,https://www.ambito.com/politica/el-mensaje-jav...,politica
4,Ámbito Financiero,2025-01-01,El ajuste alcanzó a las fiestas: las ventas de...,El 2024 fue un año marcado por un fuerte recor...,https://www.ambito.com/economia/el-ajuste-alca...,economia


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15244 entries, 0 to 15243
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   diario     15244 non-null  object        
 1   fecha      15244 non-null  datetime64[ns]
 2   titulo     15244 non-null  object        
 3   contenido  15207 non-null  object        
 4   url        15244 non-null  object        
 5   seccion    15244 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 714.7+ KB


In [39]:
# taking a sample of 50 news
sample = 50

df_sample = df.sample(sample, random_state=42).reset_index(drop=True)

In [40]:
df_sample

,diario,fecha,titulo,contenido,url,seccion
0,Ámbito Financiero,2025-04-15,El gobierno de Javier Milei eliminó el RUCA y ...,La Secretaría de Agricultura resolvió dar de b...,https://www.ambito.com/economia/el-gobierno-ja...,economia
1,La Nación,2025-03-03,Por qué la polarización favorece a Kicillof co...,El analista Rosendo Fraga examinó este domingo...,https://www.lanacion.com.ar/politica/por-que-l...,Política
2,La Nación,2025-02-25,Banco Santander le compró a Grupo Galicia el 5...,"Banco Santander SA compró el 50% de Nera, la p...",https://www.lanacion.com.ar/economia/campo/ban...,Campo
3,Ámbito Financiero,2025-04-07,Peligra en Diputados la sesión por $LIBRA: la ...,La oposición en Diputados aúna fuerzas para po...,https://www.ambito.com/politica/peligra-diputa...,politica
4,La Nación,2025-01-29,La baja de retenciones podría ayudar a contene...,La reducción transitoria de las retenciones al...,https://www.lanacion.com.ar/economia/esquema-8...,Economía
5,La Nación,2025-01-14,"Milei ganó el ""Premio Nobel Judío"" por su apoy...",El presidente Javier Milei ganó este martes el...,https://www.lanacion.com.ar/politica/milei-gan...,Política
6,Clarín,2025-04-05,Qué historia llevó a Bill Clinton a expandir e...,“Quiero un decreto para el viernes que retire ...,https://www.clarin.com/economia/historia-llevo...,economia
7,Ámbito Financiero,2025-03-25,DNU sin necesidad ni urgencia,El DNU 179/2025 que envió el Poder Ejecutivo a...,https://www.ambito.com/opiniones/dnu-necesidad...,opinion
8,Ámbito Financiero,2025-03-02,"Facundo Manes: ""Frenemos antes de que sea tard...",El diputado nacional de Democracia para Siempr...,https://www.ambito.com/politica/facundo-manes-...,politica
9,Clarín,2025-01-29,En qué sectores se produjo la caída del 75% de...,La inversión pública ejecutada durante 2024 as...,https://www.clarin.com/economia/sectores-produ...,economia


### OpenAI Requests

Initiating OpenAI client

In [41]:
try:
    with open(project_path+'.secrets/openai_api_key.txt', 'r') as file:
        key = file.read()
except FileNotFoundError:
    print("Error: The file 'openai_api_key.txt' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

client = OpenAI(api_key = key)

Structure request

In [42]:
# ---------------------------
# DEFINICIONES DE CONVENCIÓN
# ---------------------------
# Escalas:
# - Señales y sentimientos: [-1..1]  (dirección y magnitud a la vez; 0 = neutro)
# - Confianza y calidad:    [0..1]
#
# Señales de mercado (signo):
# - merval: + sube índice; - baja índice
# - fx_usdars: + se deprecia ARS (sube USD/ARS); - se aprecia ARS (baja USD/ARS)
# - tasa_bcra: + sube tasa de política; - baja tasa
# - bonos_soberanos: + sube precio (mejora riesgo / bajan rendimientos); - baja precio
# - actividad_economica: + mayor actividad; - menor actividad
#
# Reglas de simplicidad (para gpt-5-nano):
# - Si el artículo no da evidencia clara → devolver 0.0 (neutro) y/o "unknown".
# - horizonte_dias solo si aparece explícito (fechas, “en X días/meses”, “desde hoy”).
# - Listas sin duplicados; tickers en MAYÚSCULAS; strings en minúsculas salvo nombres propios.
# - Redondeo de floats a 2 decimales recomendado en el post-procesamiento.

class SenalesMercado(BaseModel):
    merval: float = Field(description="[-1..1] Sesgo esperado para el índice Merval (↑:+, ↓:-).")
    fx_usdars: float = Field(description="[-1..1] Sesgo para el tipo de cambio USD/ARS (↑:+ deprecia ARS, ↓:- aprecia ARS).")
    tasa_bcra: float = Field(description="[-1..1] Sesgo para la tasa de política del BCRA (↑:+, ↓:-).")
    bonos_soberanos: float = Field(description="[-1..1] Sesgo para el precio de bonos soberanos (↑:+ precio, ↓:- precio).")
    actividad_economica: float = Field(description="[-1..1] Sesgo sobre nivel de actividad (↑:+, ↓:-).")

class Sentimientos(BaseModel):
    valencia_general: float = Field(description="[-1..1] Tono global del artículo sobre economía/mercados.")
    gobernanza: float = Field(description="[-1..1] Tono respecto a gobierno/gobernanza (institucional/político).")
    expectativa_macro_corto: float = Field(description="[-1..1] Expectativa macro a 1–3 meses.")
    expectativa_macro_largo: float = Field(description="[-1..1] Expectativa macro a >6 meses.")
    expectativa_fin_corto: float = Field(description="[-1..1] Condiciones financieras a 1–3 meses (tasas, crédito, riesgo).")
    expectativa_fin_largo: float = Field(description="[-1..1] Condiciones financieras a >6 meses.")

class Tematicas(BaseModel):
    menciona_inflacion: bool = Field(description="Se menciona inflación/precios.")
    menciona_pbi: bool = Field(description="Se menciona PBI/crecimiento.")
    menciona_reservas: bool = Field(description="Se mencionan reservas del BCRA.")
    menciona_embi: bool = Field(description="Se menciona riesgo país/EMBI.")
    menciona_deuda: bool = Field(description="Se menciona deuda pública/privada.")
    menciona_fmi: bool = Field(description="Se menciona FMI/acuerdos.")
    menciona_salarios_paritarias: bool = Field(description="Se mencionan salarios/paritarias.")
    menciona_tipo_cambio: bool = Field(description="Se menciona tipo de cambio/dólar.")
    menciona_confianza_consumidor: bool = Field(description="Se menciona índice/sentimiento de confianza del consumidor.")

class Entidades(BaseModel):
    tipo_actor_principal: Literal[
        "gobierno_nacional","bcra","provincia","municipio",
        "empresa_local","empresa_extranjera",
        "sindicato","poder_judicial","congreso","organismo_internacional",
        "desconocido"
    ] = Field(description="Actor dominante al que refiere la noticia.")
    nombre_actor_principal: Optional[str] = Field(default="unknown", description="Nombre del actor (si es claro).")
    empresas_mencionadas: List[str] = Field(default_factory=list, description="Empresas citadas (nombre legal).")
    tickers_mencionados: List[str] = Field(default_factory=list, description="Tickers (BYMA/ADRs) en MAYÚSCULAS, sin duplicados.")
    sectores_mencionados: List[str] = Field(default_factory=list, description="Sectores/industrias relevantes (ej.: bancos, energía).")

class Evento(BaseModel):
    tipo_evento: Literal[
        "monetario","fiscal","regulatorio","corporativo","externo",
        "sindical_social","judicial","electoral","otro","desconocido"
    ] = Field(description="Categoría simple del hecho principal.")
    shock: Literal["positivo","negativo","mixto","neutro","desconocido"] = Field(description="Signo cualitativo del shock.")
    caracter: Literal["retroactivo","vigente","prospectivo","desconocido"] = Field(description="Temporalidad legal/efectiva.")
    horizonte_dias: Optional[int] = Field(default=None, description="Días hasta impacto si aparece explícito; si no, null.")

class CalidadFuente(BaseModel):
    categoria: Literal["oficial","periodistica","rumor","analisis","desconocido"] = Field(
        description="Tipo de fuente del contenido."
    )
    score: float = Field(description="[0..1] Confiabilidad de la fuente según categoría (regla guía en prompt).")

class NewFeatures(BaseModel):
    # Núcleo
    entidades: Entidades
    evento: Evento
    mercado: SenalesMercado
    sentimientos: Sentimientos
    tematicas: Tematicas

    # Calidad / confianza
    calidad_fuente: CalidadFuente
    confianza: float = Field(description="[0..1] Confianza global de extracción (claridad y evidencia).")

### SYSTEM_PROMPT

SYSTEM_PROMPT = '''
Eres un analista económico-financiero especializado en Argentina.
Objetivo: extraer datos ESTRUCTURADOS de una noticia para modelar el MERVAL.
Reglas de oro (optimizado para modelos pequeños):
- Usa SOLO el texto de la noticia. NO infieras más allá.
- Si la evidencia no es clara: usa 0.0 (neutro), "unknown" o null (horizonte).
- Señales y sentimientos son valores en [-1..1] (dirección y magnitud a la vez).
- Confianza y calidad de fuente en [0..1].
- “horizonte_dias” SOLO si hay mención explícita (“en X días/meses”, “desde hoy”, una fecha clara).
- Tickers en MAYÚSCULAS; listas sin duplicados.
- No cites fuentes externas.

Guías rápidas:
- Calidad de fuente → score por defecto:
  oficial: 0.9, periodistica: 0.7, analisis: 0.6, rumor: 0.3, desconocido: 0.5
- Señales de mercado (signo):
  merval (+ sube índice), fx_usdars (+ se deprecia ARS), tasa_bcra (+ sube tasa),
  bonos_soberanos (+ sube precio), actividad_economica (+ sube actividad).
- Si el artículo es de temática no económica/financiera: deja señales en 0.0, evento=“desconocido”, confianza ≤ 0.3.

Salida: JSON ESTRICTO que valide contra el esquema. No agregues texto extra.
'''
### USER_TEMPLATE (sin pasos “pesados”, directo al punto)

USER_TEMPLATE = '''
Diario: {diario}
Fecha: {fecha}  # formato YYYY-MM-DD
Seccion: {seccion}
Titulo: {titulo}
Contenido: {contenido}

Devuelve SOLO el JSON con el esquema pedido (sin texto adicional).
'''


In [43]:
model = "gpt-5-nano"

results = []
for _, row in df_sample.iterrows():
    print(f"Processing row {_}/{len(df_sample)}")
    # Manejo de faltantes
    contenido = (row.get("contenido") or "")[:8000]
    prompt = USER_TEMPLATE.format(
        diario=row.get("diario", "unknown"),
        fecha=str(row.get("fecha", "unknown")),
        seccion=row.get("seccion", "unknown"),
        titulo=row.get("titulo", "unknown"),
        contenido=contenido,
        url=row.get("url", "unknown"),
    )

    resp = client.responses.parse(
        model=model,
        input=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
        text_format=NewFeatures,
    )

    parsed = resp.output_parsed
    results.append(parsed.model_dump())

df_features = json_normalize(results, sep="__")
df_sample_features = pd.concat([df_sample.reset_index(drop=True), df_features], axis=1)

Processing row 0/50
Processing row 1/50
Processing row 2/50
Processing row 3/50
Processing row 4/50
Processing row 5/50
Processing row 6/50
Processing row 7/50
Processing row 8/50
Processing row 9/50
Processing row 10/50
Processing row 11/50
Processing row 12/50
Processing row 13/50
Processing row 14/50
Processing row 15/50
Processing row 16/50
Processing row 17/50
Processing row 18/50
Processing row 19/50
Processing row 20/50
Processing row 21/50
Processing row 22/50
Processing row 23/50
Processing row 24/50
Processing row 25/50
Processing row 26/50
Processing row 27/50
Processing row 28/50
Processing row 29/50
Processing row 30/50
Processing row 31/50
Processing row 32/50
Processing row 33/50
Processing row 34/50
Processing row 35/50
Processing row 36/50
Processing row 37/50
Processing row 38/50
Processing row 39/50
Processing row 40/50
Processing row 41/50
Processing row 42/50
Processing row 43/50
Processing row 44/50
Processing row 45/50
Processing row 46/50
Processing row 47/50
Pr

Took 26 minutes for 50 news

In [44]:
df_sample_features

,diario,fecha,titulo,contenido,url,seccion,confianza,entidades__tipo_actor_principal,entidades__nombre_actor_principal,entidades__empresas_mencionadas,entidades__tickers_mencionados,entidades__sectores_mencionados,evento__tipo_evento,evento__shock,evento__caracter,evento__horizonte_dias,mercado__merval,mercado__fx_usdars,mercado__tasa_bcra,mercado__bonos_soberanos,mercado__actividad_economica,sentimientos__valencia_general,sentimientos__gobernanza,sentimientos__expectativa_macro_corto,sentimientos__expectativa_macro_largo,sentimientos__expectativa_fin_corto,sentimientos__expectativa_fin_largo,tematicas__menciona_inflacion,tematicas__menciona_pbi,tematicas__menciona_reservas,tematicas__menciona_embi,tematicas__menciona_deuda,tematicas__menciona_fmi,tematicas__menciona_salarios_paritarias,tematicas__menciona_tipo_cambio,tematicas__menciona_confianza_consumidor,calidad_fuente__categoria,calidad_fuente__score
0,Ámbito Financiero,2025-04-15,El gobierno de Javier Milei eliminó el RUCA y ...,La Secretaría de Agricultura resolvió dar de b...,https://www.ambito.com/economia/el-gobierno-ja...,economia,0.65,gobierno_nacional,Gobierno Nacional,[],[],"[agroindustria, carnico, lacteo]",regulatorio,positivo,vigente,21.0,0.0,0.0,0.0,0.0,0.0,0.40,0.50,0.20,0.15,0.20,0.25,False,False,False,False,False,False,False,False,False,periodistica,0.7
1,La Nación,2025-03-03,Por qué la polarización favorece a Kicillof co...,El analista Rosendo Fraga examinó este domingo...,https://www.lanacion.com.ar/politica/por-que-l...,Política,0.20,provincia,Axel Kicillof,[],[],[],desconocido,neutro,desconocido,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,False,False,False,False,False,False,False,False,False,periodistica,0.7
2,La Nación,2025-02-25,Banco Santander le compró a Grupo Galicia el 5...,"Banco Santander SA compró el 50% de Nera, la p...",https://www.lanacion.com.ar/economia/campo/ban...,Campo,0.80,empresa_extranjera,Banco Santander SA,"[Banco Santander SA, Grupo Galicia, Nera, Banc...",[],"[agropecuario, financiero, tecnología]",corporativo,neutro,vigente,NaN,0.0,0.0,0.0,0.0,0.0,0.30,0.20,0.10,0.00,0.20,0.25,False,False,False,False,False,False,False,False,False,periodistica,0.7
3,Ámbito Financiero,2025-04-07,Peligra en Diputados la sesión por $LIBRA: la ...,La oposición en Diputados aúna fuerzas para po...,https://www.ambito.com/politica/peligra-diputa...,politica,0.25,desconocido,None,[],[],[],desconocido,desconocido,desconocido,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,False,False,False,False,False,False,False,False,False,periodistica,0.7
4,La Nación,2025-01-29,La baja de retenciones podría ayudar a contene...,La reducción transitoria de las retenciones al...,https://www.lanacion.com.ar/economia/esquema-8...,Economía,0.80,gobierno_nacional,Gobierno Nacional,"[Portfolio Personal de Inversiones (PPI), Ecol...",[],"[agropecuario, finanzas]",regulatorio,positivo,vigente,152.0,0.0,-0.4,0.0,0.0,0.2,0.15,0.20,0.25,0.00,0.20,0.10,False,False,True,False,False,False,False,True,False,periodistica,0.7
5,La Nación,2025-01-14,"Milei ganó el ""Premio Nobel Judío"" por su apoy...",El presidente Javier Milei ganó este martes el...,https://www.lanacion.com.ar/politica/milei-gan...,Política,0.65,gobierno_nacional,Javier Milei,[],[],[],desconocido,desconocido,desconocido,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,False,False,False,False,False,False,False,False,False,periodistica,0.7
6,Clarín,2025-04-05,Qué historia llevó a Bill Clinton a expandir e...,“Quiero un decreto para el viernes que retire ...,https://www.clarin.com/economia/historia-llevo...,economia,0.55,gobierno_nacional,Donald Trump,[],[],[],regulatorio,negativo,retroactivo,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,False,False,False,False,True,False,False,False,False,periodistica,0.7
7,Ámbito Financiero,2025-03-25,DNU sin necesidad ni urgencia,El DNU 179/2025 que envió el Poder Ejecutivo a...,https://www.ambito.com/opiniones/dnu-necesidad...,opinion,0.65,congreso,None,[],

In [45]:
df_sample_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 38 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   diario                                    50 non-null     object        
 1   fecha                                     50 non-null     datetime64[ns]
 2   titulo                                    50 non-null     object        
 3   contenido                                 50 non-null     object        
 4   url                                       50 non-null     object        
 5   seccion                                   50 non-null     object        
 6   confianza                                 50 non-null     float64       
 7   entidades__tipo_actor_principal           50 non-null     object        
 8   entidades__nombre_actor_principal         44 non-null     object        
 9   entidades__empresas_mencionadas   

In [46]:
df_sample_features.to_csv(project_path+f"5-LLMs/openai/df_sample{sample}_{model}.csv", index=False)

To decide and improve:

- Avoiding quotes and other kind of feature requirend using more output tokens
- Avoiding reducing too complex features
- Aggregating news by day / by newspaper
